In [31]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
import tensorflow as tf
from keras import backend as K
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from skimage.morphology import label
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
import random
from u_net import Unet, get_unet_256
from skimage import io

In [32]:
def make_df(train_path, test_path, mask_path, img_size):
    train_ids = next(os.walk(train_path))[2]
    test_ids = next(os.walk(test_path))[2]
    X_train = np.zeros((len(train_ids), img_size, img_size, 3), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), img_size, img_size, 1), dtype=np.bool)
    for i, id_ in enumerate(train_ids):
        path = train_path + '/' + id_
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        X_train[i] = img

        m_path = mask_path + '/' + id_
        msk = io.imread(m_path, plugin='pil')
        # msk = cv2.imread(m_path, 0)
        msk = cv2.resize(msk, (img_size, img_size))
        msk = msk[..., np.newaxis]
        #msk = np.expand_dims(msk,axis = -1)
        Y_train[i] = msk

    X_test = np.zeros((len(test_ids), img_size, img_size, 3), dtype=np.uint8)
    # sizes_test = []
    for i, id_ in enumerate(test_ids):
        path = test_path + '/' + id_
        imgTest = cv2.imread(path)
        imgTest = cv2.resize(imgTest, (img_size, img_size))
        X_test[i] = imgTest

    return X_train, Y_train, X_test

In [33]:
def generator(xtr, xval, ytr, yval, batch_size):
    data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=90.,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         fill_mode = "constant",
                         cval = 0.)

    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    image_datagen.fit(xtr, seed=7)
    mask_datagen.fit(ytr, seed=7)
    image_generator = image_datagen.flow(xtr, batch_size=batch_size, seed=7)
    mask_generator = mask_datagen.flow(ytr, batch_size=batch_size, seed=7)
    train_generator = zip(image_generator, mask_generator)

    val_gen_args = dict()
    image_datagen_val = ImageDataGenerator(**val_gen_args)
    mask_datagen_val = ImageDataGenerator(**val_gen_args)
    image_datagen_val.fit(xval, seed=7)
    mask_datagen_val.fit(yval, seed=7)
    image_generator_val = image_datagen.flow(xval, batch_size=batch_size, seed=7)
    mask_generator_val = mask_datagen.flow(yval, batch_size=batch_size, seed=7)
    val_generator = zip(image_generator_val, mask_generator_val)

    return train_generator, val_generator

In [34]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec))

In [35]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

In [36]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [37]:
def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [44]:
def pred():
    preds_test = model.predict(X_test, verbose=1)
    preds_test_t = (preds_test > 0.5).astype(np.uint8)

    preds_test_upsampled = []
    for i in range(len(preds_test)):
        preds_test_upsampled.append(cv2.resize(preds_test[i],
                                               (sizes_test[i][0], sizes_test[i][1])).transpose(1, 0))

    test_ids = next(os.walk(test_path))[1]
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
    sub = pd.DataFrame()
    sub['ImageId'] = new_test_ids
    sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    sub.to_csv('sub_aug_512.csv', index=False)


if __name__ == "__main__":
    img_size = 256
    batch_size =4
    train_path = 'Input/train'
    test_path = 'Input/test'
    mask_path = 'Input/mask'

    X_train, Y_train, X_test = make_df(train_path, test_path, mask_path, img_size)
    xtr, xval, ytr, yval = train_test_split(X_train, Y_train, test_size=0.1, random_state=7)
    train_generator, val_generator = generator(xtr, xval, ytr, yval, batch_size)

    #model = Unet(img_size)
    model = get_unet_256(input_shape=(img_size, img_size, 3),num_classes=1)
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[mean_iou, dice_coeff])
    callbacks = [EarlyStopping(monitor='val_mean_iou',
                               patience=50,
                               verbose=2,
                               min_delta=1e-4,
                               mode='max'
                                ),
                 ReduceLROnPlateau(monitor='val_mean_iou',
                                   factor=np.sqrt(0.1),
                                   patience=5,
                                   cooldown=2,
                                   verbose=1,
                                   epsilon=1e-4,
                                   mode='max'
                                    ),
                 ModelCheckpoint(monitor='val_mean_iou',
                                 filepath='weights/unet_256_arghya.{epoch:02d}-{val_loss:.5f}-{val_mean_iou:.5f}.h5',
                                 save_best_only=True,
                                 save_weights_only=False,
                                 mode='max',
                                 #period = 1
                                  ),
                 TensorBoard(log_dir="logs/",  histogram_freq=0,  write_graph=True, write_images=True)]

    model.load_weights('weights/unet_256_arghya.272-0.03398-0.96499.h5')
    print('weights loaded!')
    
    preds_test = model.predict(X_test)
    preds_test_t = (preds_test > 0.5).astype(np.uint8)
    ix = range(0, 6)
    d= 1
    for item in ix:
        imgTest = X_test[item]
        mask_in = (preds_test_t[item]).astype(np.uint8)
        mask_cleaned = (mask_in * 255).astype(np.uint8)
        mask_cleaned = np.squeeze(mask_cleaned)
        mask_color = cv2.cvtColor(mask_cleaned, cv2.COLOR_GRAY2BGR)
        img_masked = cv2.addWeighted(imgTest, 0.5, mask_color, 0.5, 0)
        filename = "heat_1100_kwm2_8bit_%3.5d.tif" %d
        cv2.imwrite(filename, mask_color)
        d+=1



F:\Running_file\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


weights loaded!
